## Imports

In [55]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten

In [56]:
df = pd.read_csv('SoFla Neighborhoods New Construction - DT_FTL_map.csv')

In [57]:
df.columns

Index(['address', 'stories', 'units', 'dt_fl_developer', 'trd_developer',
       'developer_for_map', 'description', 'status', 'story link',
       'useful_info'],
      dtype='object')

## Image URL collector

In [58]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)


In [59]:
df['image_links'] = ''
links = []
for url in df['story link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [60]:
links

['https://s12.therealdeal.com/trd/m/up/2022/01/main_MIA_KushnerFLL-705x439.jpg',
 'https://s13.therealdeal.com/trd/m/up/2019/01/1200-bh3-650x405.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/01/main_MIA_KushnerFLL-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/06/main_MIA_BK-FTL-project-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2018/02/Ricardo-Vadia-600-650x377.jpg',
 'https://s13.therealdeal.com/trd/m/up/2019/01/1200-bh3-650x405.jpg',
 'https://s11.therealdeal.com/trd/m/up/2020/06/oko-1-650x405.jpg',
 'https://s13.therealdeal.com/trd/m/up/2021/10/MIA-Hines-buys-dev-site-in-downtown-Fort-Lauderdales-FATVillage-for-57M-MAIN-705x439.jpg',
 'https://therealdeal.com/miami/wp-content/uploads/2018/03/Rendering-of-River-Lofts-on-Fifth600-650x377.jpg',
 'https://s13.therealdeal.com/trd/m/up/2021/12/main_MIA_808-SE-4th-705x439.jpg',
 'https://s12.therealdeal.com/trd/m/up/2022/06/main-Aimco-drops-64M-for-mixed-use-dev-site-in-Fort-Lauderdales-Flagler-Village-705x439.jpg

In [61]:
df.description

0     Apartment giants Kushner Companies and Aimco p...
1     Traina Companies, a co-developer of Privé at I...
2     Apartment giants Kushner Companies and Aimco p...
3     Brooklyn-based Dependable Equities plans a 1.5...
4     Ricardo Vadia, who leads Miami-based ROVR deve...
5     Traina Companies, a co-developer of Privé at I...
6     Billionaire developer Vladislav Doronin assemb...
7     Houston-based Hines paid $57.4 million for a m...
8     Moderno Development Group, led by Christopher ...
9     Boca Raton-based Roberts Equities LLC has appr...
10    Aimco bought a redevelopment site in Flagler V...
11    Developer Dev Motwani paid $14 million for a d...
12    Jean Francois Roy, founder and CEO of Fort Lau...
13    Hollywood-based developer Walker & Dunlop are ...
14    Urban Street Development, led by real estate d...
15    Jorge Pérez’s Related Group, armed with an $84...
Name: description, dtype: object

In [62]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/miami/2022/01...
1     <a href="https://therealdeal.com/miami/2019/01...
2     <a href="https://therealdeal.com/miami/2022/01...
3     <a href="https://therealdeal.com/miami/2022/06...
4     <a href="https://therealdeal.com/miami/2018/02...
5     <a href="https://therealdeal.com/miami/2019/01...
6     <a href="https://therealdeal.com/miami/2020/06...
7     <a href="https://therealdeal.com/miami/2021/10...
8     <a href="https://therealdeal.com/miami/2018/03...
9     <a href="https://therealdeal.com/miami/2021/12...
10    <a href="https://therealdeal.com/miami/2022/06...
11    <a href="https://therealdeal.com/miami/2020/09...
12    <a href="https://therealdeal.com/miami/2021/06...
13    <a href="https://therealdeal.com/miami/2019/07...
14    <a href="https://therealdeal.com/miami/2020/05...
15    <a href="https://therealdeal.com/miami/2021/08...
Name: description_link, dtype: object

## Google Maps API Geolocater Setup

In [63]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [64]:
df['geo_address'] = df['address'] + ', Fort Lauderdale, FL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## HTML popup formatter

In [65]:
def popup_html(row):
    i = row
    address = df['address'].iloc[i]
    developer = df['developer_for_map'].iloc[i]
    description = df['description_link'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Address: <strong>{}'''.format(address) + '''</strong><br>
    Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>
    <strong>{}'''.format(description) + '''</strong></html>
    '''
    return html


## Map Maker

In [66]:
df.description_link

0     <a href="https://therealdeal.com/miami/2022/01...
1     <a href="https://therealdeal.com/miami/2019/01...
2     <a href="https://therealdeal.com/miami/2022/01...
3     <a href="https://therealdeal.com/miami/2022/06...
4     <a href="https://therealdeal.com/miami/2018/02...
5     <a href="https://therealdeal.com/miami/2019/01...
6     <a href="https://therealdeal.com/miami/2020/06...
7     <a href="https://therealdeal.com/miami/2021/10...
8     <a href="https://therealdeal.com/miami/2018/03...
9     <a href="https://therealdeal.com/miami/2021/12...
10    <a href="https://therealdeal.com/miami/2022/06...
11    <a href="https://therealdeal.com/miami/2020/09...
12    <a href="https://therealdeal.com/miami/2021/06...
13    <a href="https://therealdeal.com/miami/2019/07...
14    <a href="https://therealdeal.com/miami/2020/05...
15    <a href="https://therealdeal.com/miami/2021/08...
Name: description_link, dtype: object

In [67]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'New Development in Downtown Fort Lauderdale')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [68]:
m.save('index.html')